In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from numpy.linalg import inv
import scipy.special
%matplotlib inline

In [ ]:
def phi(x):
    # Parameter Estimation and Inverse Problems, 3rd edition, 2018
    # by R. Aster, B. Borchers, C. Thurber
    # z=phi(x)
    #
    # Calculates the normal distribution and returns the value of the
    # integral
    #
    #       z=int((1/sqrt(2*pi))*exp(-t^2/2),t=-infinity..x)
    #
    # Input Parameters:
    #   x - endpoint of integration (scalar)
    3
    # Output Parameters:
    #   z - value of integral
    #Python version coded by Yuan Tian @UAF 2021
    if (x >= 0):
        z = 0.5 + 0.5 * scipy.special.erf(x/np.sqrt(2))
    else:
        z = 1 - phi(-x)
    return z
def chi2cdf(x,m):
    # Parameter Estimation and Inverse Problems, 3rd edition, 2018
    # by R. Aster, B. Borchers, C. Thurber
    # p=chi2cdf(x,m)
    #
    # Computes the Chi^2 CDF, using a transformation to N(0,1) on page
    # 333 of Thistead, Elements of Statistical Computing.
    #
    # Input Parameters:
    #   x - end value of chi^2 pdf to integrate to. (scalar)
    #   m - degrees of freedom (scalar)
    #
    # Output Parameters:
    #   p - probability that Chi^2 random variable is less than or
    #       equal to x (scalar).
    #
    # Note that x and m must be scalars.

    #Python version coded by Yuan Tian @UAF 2021
    if x==(m-1):
        p=0.5
    else:
        z = (x - m + 2 / 3 - 0.08 / m) * np.sqrt((m - 1)*np.log((m - 1)/x)+x-(m - 1)) / np.abs(x-m+1);
        p = phi(z)
    return p
def chi2inv(p,nu):
    # Parameter Estimation and Inverse Problems, 3rd edition, 2018
    # by R. Aster, B. Borchers, C. Thurber
    # x=chi2inv(p,nu)
    #
    # Computes the inverse Chi^2 distribution corresponding to a given
    # probability that a Chi^2 random variable with the given degrees
    # of freedom is less than or equal to x.  Uses chi2cdf.m.
    #
    # Input Parameters:
    #   p - probability that Chi^2 random variable is less than or
    #       equal to x (scalar).
    #   nu - degrees of freedom (scalar)
    #
    # Output Parameters:
    #   x - corresponding value of x for given probability.
    #
    # Note that x and m must be scalars.
    # Special cases.
    #Python version coded by Yuan Tian @UAF 2021
    if (p >= 1.0):
        return np.inf
    elif (p == 0.0):
        return 0
    elif (p < 0):
        return -np.inf

    # find a window with a cdf containing p
    l = 0.0
    r = 1.0
    
    while (chi2cdf(r, nu) < p):
        l = r
        r = r * 2
    # do a binary search until we have a sufficiently small interval around x
    while (((r - l) / r) > 1.0e-5):
        m = (l + r) / 2
        if (chi2cdf(m, nu) > p):
            r = m
        else:
            l = m
    return (l+r)/2

In [ ]:
dat = scipy.io.loadmat('data/data1.mat')

In [ ]:
data1=dat['data1']
t=data1[:,0]
y=data1[:,1]
sigma=data1[:,2]
N=len(t)

print('displaying t,y,sigma')
[t , y , sigma ]
# Build the parabolic system matrix

G = np.concatenate(((np.ones((N,1)), t.reshape((N, 1)), -0.5*t.reshape((N, 1))**2)),axis=1)

# Apply the weighting

yw = (y/sigma).reshape((N, 1))
Gw = G/np.concatenate((sigma.reshape((N, 1)),sigma.reshape((N, 1)),sigma.reshape((N, 1))),axis=1)

# Solve for the least-squares solution
print('Least-squares solution')
m = inv(Gw.T@Gw)@Gw.T@yw

# Get the covariance matrix
ginv = inv(Gw.T@Gw)@Gw.T

print('Covariance matrix')
covm = ginv@ginv.T

# Get the 1.96-sigma (95%) conf intervals
print('95% parameter confidence intervals (m-, mest, m+)')
delm = 1.96*np.sqrt(np.diag(covm)).reshape((3, 1))
[m-delm , m , m+delm]
# Because there are 3 parameters to estimate, we have N-3 degrees
# of freedom.
#
dof = N-3
print('Chi-square misfit for %.2f dof' %dof)
chi2 = np.linalg.norm((y.reshape((N, 1)) - G@m)/sigma.reshape((N, 1)))**2

# Find the p-value for this data set
print('chi-square p-value')
p = 1-chi2cdf(chi2,dof)

# Find the parameter correlations
s=np.sqrt(np.diag(covm)).reshape((3, 1))

print('correlation matrix')
r = covm/(s@s.T)


# Plot the data and model predicted data
xx=np.arange(np.min(t)-1,np.max(t)+1+0.05,0.05)
mm=m[0][0]+m[1][0]*xx-0.5*m[2][0]*xx**2


In [ ]:
plt.figure(figsize=(9,7))
plt.plot(xx,mm,'k')
#plt.plot(t,y,'o-')
#plt.errorbar(t,y,yerr=sigma,marker='s',mfc='k',ms=10)
plt.errorbar(t,y,yerr=sigma, ls='none',marker='o',mfc='none',capsize=3,mec='k',ms=3)
plt.xlabel('Time (s)');
plt.ylabel('Elevation (m)')




In [ ]:
print('Displaying Data and Model Fit (fig 1)')
# Output covm and the eigenvalues/eigenvectors of covm.
print('Covariance matrix for fitted parameters.')
print('Eigenvalues/eigenvectors of the covariance matrix');
lam,u=np.linalg.eig(inv(covm))
print('95% confidence ellipsoid semiaxis lengths')
semi_axes = [np.sqrt(chi2inv(0.95,3)*(1/lam))]
print('95% confidence ellipsoid semiaxes')
# Monte Carlo Section
y0 = G@m
mmc=np.zeros((3,1000))
chimc=[]
for nreal in  range(1000):
  # Generate a trial data set of perturbed, weighted data
  ytrial = y0.reshape((N,1))+sigma.reshape((N,1))*np.random.randn(N,1)
  ywtrial=ytrial/sigma.reshape((N,1))
  mmc[:,nreal]=(np.linalg.pinv(Gw)@ywtrial).flatten()
  chimc.append(np.linalg.norm((G@mmc[:,nreal]-ytrial.flatten())/sigma)**2)

mmc=mmc.T

In [ ]:
# Plot the histogram of chi squared values
plt.figure()
plt.hist(chimc,30)
plt.ylabel('N');
plt.xlabel(r'$\chi^2$');
print('Displaying 1000 Monte-Carlo Chi-square Values (fig 2)')


In [ ]:
# Plot the histograms of the model parameters
plt.figure(figsize=(10,5))

plt.subplot(1,3,1)
plt.hist(mmc[:,0])
plt.title(r'$m_1 (m)$')


plt.subplot(1,3,2)

plt.hist(mmc[:,1])
plt.title(r'$m_2 (m/s)$')


plt.subplot(1,3,3)
plt.hist(mmc[:,2])
plt.title(r'$m_3 (m/s^2)$')
print('Displaying Monte-Carlo Model Histograms (fig 3)')

In [ ]:
# Plot the realizations of each pair of model parameters with the other
plt.figure(figsize=(10,5))

plt.subplot(1,3,1)
plt.plot(mmc[:,0],mmc[:,1],'k*')
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_2 (m/s)$')


plt.subplot(1,3,2)

plt.plot(mmc[:,0],mmc[:,2],'k*')
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_3 (m/s)$')


plt.subplot(1,3,3)

plt.plot(mmc[:,1],mmc[:,2],'k*')
plt.xlabel(r'$m_2 (m)$')
plt.ylabel(r'$m_3 (m/s)$')


print('Displaying Projections of 1000 Monte-Carlo models (fig 4)')

In [ ]:
# Plot the 95% error ellipses for each pair of parameters
# Note that because we're doing pairs of parameters there are 2
# degrees of freedom in the Chi-square here, rather than 3.  
#
#generate a vector of angles from 0 to 2*pi
theta=np.arange(0,2*np.pi,0.01)
delta=np.sqrt(chi2inv(0.95,2))
#the radii in each direction from the center
r=np.zeros((len(theta),2))


# compute the data for the m1, m2 ellipsoid.
C=covm[:2,:2]
[lam,u]=np.linalg.eig(inv(C))
lam=np.diag(lam)
#calculate the x component of the ellipsoid for all angles
r[:,0]=(delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1]=(delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)
plt.figure(figsize=(13,6))
# plot the data for the m1, m2 ellipsoid
plt.subplot(1,3,1)
plt.plot(m[0]+r[:,0],m[1]+r[:,1],'k');
plt.fill(m[0]+r[:,0],m[1]+r[:,1],'k');
#axis([-50 50 85 110]);
plt.xlabel(r'$m_1 (m)$');
plt.ylabel(r'$m_2 (m/s)$');

# compute the data for the m1, m3 ellipsoid.
indc=np.ix_([0, 2], [0,2])
C=covm[indc]
lam,u=np.linalg.eig(inv(C))
lam=np.diag(lam)
deltachisq=chi2inv(0.95,2)
delta=np.sqrt(deltachisq)
#calculate the x component of the ellipsoid for all angles
r[:,0]=(delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1]=(delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)
plt.subplot(1,3,2)
plt.plot(m[0]+r[:,0],m[1]+r[:,1],'k');
plt.fill(m[0]+r[:,0],m[1]+r[:,1],'k');
#axis([-50 50 85 110]);
plt.xlabel(r'$m_1 (m)$');
plt.ylabel(r'$m_3 (m/s)$')

# compute the data for the m2, m3 ellipsoid.
indc=np.ix_([1, 2], [1,2])
C=covm[indc]
lam,u=np.linalg.eig(inv(C))
lam=np.diag(lam)
deltachisq=chi2inv(0.95,2)
delta=np.sqrt(deltachisq)
#calculate the x component of the ellipsoid for all angles
r[:,0]=(delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1]=(delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)
plt.subplot(1,3,3)
plt.plot(m[0]+r[:,0],m[1]+r[:,1],'k');
plt.fill(m[0]+r[:,0],m[1]+r[:,1],'k');
#axis([-50 50 85 110]);
plt.xlabel(r'$m_2 (m)$');
plt.ylabel(r'$m_3 (m/s)$')
print('Displaying 95% Confidence Ellipse Projections (fig 5)')